In [1]:
import torch
import os
from pathlib import Path
import json

/Users/michalisdikaiopoulos/Desktop/Master/Autumn2025/Advanced Topics in Deep Learning/Assignments/Assignment_2/Winning-Ticket-Hypothesis-Study/.venv/lib/python3.9/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


In [2]:
CHECKPOINTS = [
    "./models/lottery_ticket_qqp_sparsity70_seed42.pt",
    "./models/lottery_ticket_sst2_sparsity60_seed42.pt",
    "./models/ensemble_ticket_sst2_qqp_sparsity47_seed42.pt",
    "./models/lottery_ticket_qqp_sparsity0_seed42.pt",
    "./models/lottery_ticket_sst2_sparsity0_seed42.pt",
]

In [3]:
def verify_checkpoint(checkpoint_path):
    """Thoroughly inspect a checkpoint file."""

    print("\n" + "="*70)
    print(f"📦 Inspecting: {os.path.basename(checkpoint_path)}")
    print("="*70)

    if not os.path.exists(checkpoint_path):
        print(f"❌ File not found: {checkpoint_path}")
        return None

    # Load checkpoint
    try:
        ckpt = torch.load(checkpoint_path, map_location="cpu")
    except Exception as e:
        print(f"❌ Failed to load checkpoint: {e}")
        return None

    # --- Checkpoint Structure ---
    print("\n🔍 Checkpoint Keys:")
    for key in ckpt.keys():
        print(f"   - {key}")

    # --- Model State Dict ---
    if "model_state_dict" in ckpt:
        state_dict = ckpt["model_state_dict"]
        print(f"\n🧠 Model State Dict: {len(state_dict)} keys")

        # Categorize keys
        bert_keys = [k for k in state_dict.keys() if k.startswith("bert.")]
        classifier_keys = [k for k in state_dict.keys() if k.startswith("classifier")]
        other_keys = [k for k in state_dict.keys() if not k.startswith("bert.") and not k.startswith("classifier")]

        print(f"   - BERT encoder keys: {len(bert_keys)}")
        print(f"   - Classifier keys: {len(classifier_keys)}")
        if other_keys:
            print(f"   - Other keys: {len(other_keys)}")

        # Show classifier shape
        if classifier_keys:
            print(f"\n   📊 Classifier layers:")
            for key in classifier_keys:
                shape = state_dict[key].shape
                print(f"      {key}: {shape}")

        # Sample BERT weights
        print(f"\n   🔬 Sample BERT weights (first 3):")
        for key in list(bert_keys)[:3]:
            shape = state_dict[key].shape
            print(f"      {key}: {shape}")

    # --- Mask Information ---
    if "mask_dict" in ckpt:
        mask_dict = ckpt["mask_dict"]
        print(f"\n🎭 Mask Dict: {len(mask_dict)} masks found")

        # Count masked parameters
        total_params = 0
        total_nonzero = 0

        for key, mask in mask_dict.items():
            total_params += mask.numel()
            total_nonzero += mask.sum().item()

        sparsity = (1 - total_nonzero / total_params) * 100
        print(f"   - Total parameters: {total_params:,}")
        print(f"   - Non-zero parameters: {total_nonzero:,}")
        print(f"   - Sparsity: {sparsity:.2f}%")

        # Show some mask keys
        print(f"\n   🔬 Sample mask keys (first 3):")
        for key in list(mask_dict.keys())[:3]:
            shape = mask_dict[key].shape
            nonzero = mask_dict[key].sum().item()
            total = mask_dict[key].numel()
            print(f"      {key}: {shape} -> {nonzero}/{total} active ({nonzero/total*100:.1f}%)")
    else:
        print(f"\n⚠️  No mask_dict found - this might be a dense model (sparsity 0)")

    # --- Training Metadata ---
    metadata_keys = ["task", "seed", "sparsity", "final_accuracy", "best_val_acc"]
    print(f"\n📋 Training Metadata:")
    for key in metadata_keys:
        if key in ckpt:
            print(f"   - {key}: {ckpt[key]}")

    # Check for any other metadata
    other_metadata = [k for k in ckpt.keys() if k not in ["model_state_dict", "mask_dict"] + metadata_keys]
    if other_metadata:
        print(f"\n   Additional metadata:")
        for key in other_metadata:
            value = ckpt[key]
            if isinstance(value, (int, float, str, bool)):
                print(f"   - {key}: {value}")
            else:
                print(f"   - {key}: {type(value)}")

    # --- Verification Summary ---
    print(f"\n✅ Verification Summary:")
    has_encoder = "model_state_dict" in ckpt and len([k for k in ckpt["model_state_dict"].keys() if k.startswith("bert.")]) > 0
    has_classifier = "model_state_dict" in ckpt and len([k for k in ckpt["model_state_dict"].keys() if k.startswith("classifier")]) > 0
    has_mask = "mask_dict" in ckpt

    print(f"   ✓ BERT encoder weights: {'✅' if has_encoder else '❌'}")
    print(f"   ✓ Classifier weights: {'✅' if has_classifier else '❌'}")
    print(f"   ✓ Lottery ticket mask: {'✅' if has_mask else '⚠️  (dense model)'}")

    return {
        "path": checkpoint_path,
        "has_encoder": has_encoder,
        "has_classifier": has_classifier,
        "has_mask": has_mask,
        "sparsity": sparsity if has_mask else 0.0,
    }

In [4]:
def verify_all_checkpoints(checkpoint_list):
    """Verify all checkpoints and create summary report."""

    print("\n" + "🔬"*35)
    print("LOTTERY TICKET CHECKPOINT VERIFICATION")
    print("🔬"*35)

    results = []
    for ckpt_path in checkpoint_list:
        result = verify_checkpoint(ckpt_path)
        if result:
            results.append(result)

    # --- Summary Table ---
    print("\n" + "="*70)
    print("📊 SUMMARY TABLE")
    print("="*70)
    print(f"{'Checkpoint':<50} {'Encoder':<10} {'Mask':<10} {'Sparsity':<10}")
    print("-"*70)

    for result in results:
        name = os.path.basename(result["path"])
        encoder = "✅" if result["has_encoder"] else "❌"
        mask = "✅" if result["has_mask"] else "⚠️"
        sparsity = f"{result['sparsity']:.1f}%"
        print(f"{name:<50} {encoder:<10} {mask:<10} {sparsity:<10}")

    print("\n✅ Verification complete!\n")
    return results

In [5]:
def check_transfer_compatibility(source_checkpoint, target_task):
    """Check if a checkpoint is compatible for transfer to target task."""

    print(f"\n🔄 Checking transfer compatibility:")
    print(f"   Source: {os.path.basename(source_checkpoint)}")
    print(f"   Target: {target_task.upper()}")

    ckpt = torch.load(source_checkpoint, map_location="cpu")
    state_dict = ckpt.get("model_state_dict", {})

    # Check BERT encoder
    bert_keys = [k for k in state_dict.keys() if k.startswith("bert.")]
    if not bert_keys:
        print("   ❌ No BERT encoder found!")
        return False

    print(f"   ✅ BERT encoder: {len(bert_keys)} parameters")

    # Check mask (for lottery tickets)
    if "mask_dict" in ckpt:
        mask_dict = ckpt["mask_dict"]
        total = sum(m.numel() for m in mask_dict.values())
        nonzero = sum(m.sum().item() for m in mask_dict.values())
        print(f"   ✅ Lottery mask: {(1-nonzero/total)*100:.1f}% sparse")
    else:
        print(f"   ⚠️  No mask found (dense model)")

    # Check classifier (will be reinitialized anyway)
    classifier_keys = [k for k in state_dict.keys() if k.startswith("classifier")]
    if classifier_keys:
        print(f"   ℹ️  Classifier found (will be reinitialized for {target_task})")

    print(f"   ✅ Compatible for transfer learning!")
    return True

In [6]:
if __name__ == "__main__":
    # Verify all checkpoints
    results = verify_all_checkpoints(CHECKPOINTS)

    # Test a few transfer scenarios
    print("\n" + "="*70)
    print("🧪 TESTING TRANSFER SCENARIOS")
    print("="*70)

    test_scenarios = [
        ("./models/lottery_ticket_qqp_sparsity70_seed42.pt", "sst2"),
        ("./models/lottery_ticket_sst2_sparsity60_seed42.pt", "qqp"),
        ("./models/ensemble_ticket_sst2_qqp_sparsity47_seed42.pt", "sst2"),
    ]

    for source, target in test_scenarios:
        if os.path.exists(source):
            check_transfer_compatibility(source, target)


🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬
LOTTERY TICKET CHECKPOINT VERIFICATION
🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬

📦 Inspecting: lottery_ticket_qqp_sparsity70_seed42.pt

🔍 Checkpoint Keys:
   - model_state_dict
   - sparsity
   - seed
   - history

🧠 Model State Dict: 203 keys
   - BERT encoder keys: 0
   - Classifier keys: 0
   - Other keys: 203

   🔬 Sample BERT weights (first 3):

⚠️  No mask_dict found - this might be a dense model (sparsity 0)

📋 Training Metadata:
   - seed: 42
   - sparsity: 0.7

   Additional metadata:
   - history: <class 'dict'>

✅ Verification Summary:
   ✓ BERT encoder weights: ❌
   ✓ Classifier weights: ❌
   ✓ Lottery ticket mask: ⚠️  (dense model)

📦 Inspecting: lottery_ticket_sst2_sparsity60_seed42.pt

🔍 Checkpoint Keys:
   - model_state_dict
   - sparsity
   - seed
   - history

🧠 Model State Dict: 203 keys
   - BERT encoder keys: 0
   - Classifier keys: 0
   - Other keys: 203

   🔬 Sample BERT weights (first 3):

⚠️  No mask_dict found - this might be a 